# ROS Navigation

## Chapter 4: Path Planning Part 2 (Obstacle Avoidance)

<img src="img/husky_sim.png" width="1000"></img>

<p style="background:green;color:white;">SUMMARY</p>

Estimated time of completion: <b>3 hours</b><br><br>What you will learn with this unit?
<ul>
  <li>What means Path Planning in ROS Navigation?</li>
  <li>How does Path Planning work?</li>
  <li>How does the move_base node work?</li>
  <li>What is a Costmap?</li>
</ul>

<p style="background:green;color:white;">END OF SUMMARY</p>

## The Local Planner

Once the global planner has calculated the path to follow, this path is sent to the local planner. The local planner, then, will execute each segment of the global plan (let's imagine the local plan as a smaller part of the global plan). So, <b>given a plan to follow (provided by the global planner) and a map, the local planner will provide velocity commands in order to move the robot</b>. 
<br><br>
Unlike the global planner, the <b>local planner monitors the odometry and the laser data</b>, and chooses a collision free local plan (let's imagine the local plan as a smaller part of the global plan) for the robot. So, the local planner <b>can recompute the robot's path on the fly</b> in order to keep the robot from striking objects, yet still allowing it to reach its destination.
<br><br>
Once the local plan is calculated, it is published into a topic named <b>/local_plan</b>. The local planner also publishes the portion of the global plan that it is attemting to follow into the topic <b>/global_plan</b>. Let's do an exercise so you can check this better.

<p style="background:#EE9023;color:white;">Exercise 2.2</p>
<br>
a) Open Rviz and add Displays in order to be able to visualize the /global_plan and the /local_plan topics of the Local Planner.
<br><br>
b) Send a Goal Pose to the Robot and visualize both topics.
<br>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 2.2</p>

<img src="img/dwa_local_vs_global.png" width="800"></img>

<p style="background:#EE9023;color:white;">End of Exercise 2.2</p>

As for the global planner, there also exist different types of local planners. Depending on your setup (the robot you use, the environment to navigate, etc.) and the type of performance you want, you will use one or another. Let's have a look at the most important ones.

### base_local_planner

The base local planner provides implementatios of the <i>Trajectory Rollout</i> and the <i>Dynamic Window Approach (DWA)</i> algorithms in order to calculate and execute a global plan for the robot.

Summarizing, The basic idea of how this algorithms work is as follows:
<ol>
<li>Discretely sample in the robot's control space</li>
<li>For each sampled velocity, performs forward simulation from the robot's current state to predict what would happen if the sampled velocity was applied.</li>
<li>Evaluate each trajectory resulting from the forward simulation.</li>
<li>Discard illegal trajectories.</li>
<li>Pick the highest-scoring trajectory and send the associated velocities to the mobile base.</li>
<li>Rinse and Repeat.</li>
</ol>

DWA differs from Trajectory Rollout in how the robot's space is sampled. Trajectory Rollout samples from the set of achievable velocities over the entire forward simulation period given the acceleration limits of the robot, while DWA samples from the set of achievable velocities for just one simulation step given the acceleration limits of the robot.
<br><br>
DWA is a more efficient algorithm because it samples a smaller space, but may be outperformed by TRajectory Rollout for robots with low acceleration limits because DWA does not forward simulate constant accelerations. In practice, DWA and Trajectory Rollout perform similar, so <b>it's recommended to use DWA because of its efficiency gains</b>.
<br><br>
The DWA algorithm of the base local planner has been improved in a new local planner separated from this one. That's the dwa local planner we'll see next.

### dwa_local_planner

The dwa local planner provides an implementation of the <i>Dynamic Window Approach</i> algorithm. It is basically a re-write of the base local planner's DWA (Dynamic Window Approach) option, but the code is a lot cleaner and easier to understand, particularly in the way that trajectories are simulated. 
<br><br>
So, for applications that use the DWA approach for local planning, the dwa_local_planner is probaly the best choice. This is commonly the <b>most used option</b>.



### eband_local_planner

The eband local planner implements the <i>Elastic Band</i> method in order to calculate the local plan to follow.

### teb_local_planner

The teb local planner implements the <i>Timed Elastic Band</i> method in order to calculate the local plan to follow.

### Change the local planner

As for the global planner, you can also select which local planner you want to use. This is also done in the move_base node parameters file, by adding one of the following lines:

<p style="background:#EE9023;color:white;">Exercise 2.3</p>
<br>
Change the local planner used.
<br><br>
Check the differences.
<br>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 2.3</p>

<img src="img/dwa_local_vs_global.png" width="800"></img>

<p style="background:#EE9023;color:white;">End of Exercise 2.3</p>

As you would expect, the local planner also has its own parameters. This parameters will be different depending on the local planner you use. In this course, we'll be focusing on the dwa local planner parameters, since it's the most common choice. Anyways, if you want to check the specific parameters for the other local planners, you can have a look at them here:
<br><br>
base_local_planner: http://wiki.ros.org/base_local_planner
<br>
eband_local_planner: http://wiki.ros.org/eband_local_planner
<br>
teb_local_planner: http://wiki.ros.org/teb_local_planner

### dwa local planner Parameters

The parameters for the local planner are set in another YAML file. The most important parameters for the dwa local planner are the following:


#### Robot Configuration Parameters

<ul>
<li><b>/acc_lim_x (default: 2.5)</b>: The x acceleration limit of the robot in meters/sec^2</li>
<li><b>/acc_lim_y (default: 2.5)</b>: The y acceleration limit of the robot in meters/sec^2</li>
<li><b>/acc_lim_th (default: 3.2)</b>: The rotational acceleration limit of the robot in radians/sec^2</li>
<li><b>/max_trans_vel (default: 0.55)</b>: The absolute value of the maximum translational velocity for the robot in m/s</li>
<li><b>/min_trans_vel (default: 0.1)</b>: The absolute value of the minimum translational velocity for the robot in m/s</li>
<li><b>/max_vel_x (default: 0.55)</b>: The maximum x velocity for the robot in m/s.</li>
<li><b>/min_vel_x (default: 0.0)</b>: The minimum x velocity for the robot in m/s, negative for backwards motion.</li>
<li><b>/max_vel_y (default: 0.1)</b>: The maximum y velocity for the robot in m/s</li>
<li><b>/min_vel_y (default: -0.1)</b>: The minimum y velocity for the robot in m/s</li>
<li><b>/max_rot_vel (default: 1.0)</b>: The absolute value of the maximum rotational velocity for the robot in rad/s</li>
<li><b>/min_rot_vel (default: 0.4)</b>: The absolute value of the minimum rotational velocity for the robot in rad/s</li>
</ul>

#### Goal Tolerance Parameters

<ul>
<li><b>/yaw_goal_tolerance (double, default: 0.05)</b>: The tolerance in radians for the controller in yaw/rotation when achieving its goal</li>
<li><b>/xy_goal_tolerance (double, default: 0.10)</b>: The tolerance in meters for the controller in the x and y distance when achieving a goal</li>
<li><b>/latch_xy_goal_tolerance (bool, default: false)</b>: If goal tolerance is latched, if the robot ever reaches the goal xy location it will simply rotate in place, even if it ends up outside the goal tolerance while it is doing so.</li>
</ul>

#### Forward Simulation Parameters

<ul>
<li><b>/sim_time (default: 1.7)</b>: The amount of time to forward-simulate trajectories in seconds</li>
<li><b>/sim_granularity (default: 0.025)</b>: The step size, in meters, to take between points on a given trajectory</li>
<li><b>/vx_samples (default: 3)</b>: The number of samples to use when exploring the x velocity space</li>
<li><b>/vy_samples (default: 10)</b>: The number of samples to use when exploring the y velocity space</li>
<li><b>/vtheta_samples (default: 20)</b>: The number of samples to use when exploring the theta velocity space</li>
</ul>

#### Trajectory Scoring Parameters

<ul>
<li><b>/path_distance_bias (default: 32.0)</b>: The weighting for how much the controller should stay close to the path it was given</li>
<li><b>/goal_distance_bias (default: 24.0)</b>: The weighting for how much the controller should attempt to reach its local goal, also controls speed</li>
<li><b>/occdist_scale (default: 0.01)</b>: The weighting for how much the controller should attempt to avoid obstacles</li>
</ul>

Here you have an example of the dwa_local_planner_params.yaml:

In [ ]:
DWAPlannerROS:

# Robot Configuration Parameters - Kobuki
  max_vel_x: 0.5  # 0.55
  min_vel_x: 0.0

  max_vel_y: 0.0  # diff drive robot
  min_vel_y: 0.0  # diff drive robot

  max_trans_vel: 0.5 # choose slightly less than the base's capability
  min_trans_vel: 0.1  # this is the min trans velocity when there is negligible rotational velocity
  trans_stopped_vel: 0.1

  # Warning!
  #   do not set min_trans_vel to 0.0 otherwise dwa will always think translational velocities
  #   are non-negligible and small in place rotational velocities will be created.

  max_rot_vel: 5.0  # choose slightly less than the base's capability
  min_rot_vel: 0.4  # this is the min angular velocity when there is negligible translational velocity
  rot_stopped_vel: 0.4

  acc_lim_x: 1.0 # maximum is theoretically 2.0, but we
  acc_lim_theta: 2.0
  acc_lim_y: 0.0      # diff drive robot

# Goal Tolerance Parameters
  yaw_goal_tolerance: 0.3  # 0.05
  xy_goal_tolerance: 0.15  # 0.10
  # latch_xy_goal_tolerance: false

# Forward Simulation Parameters
  sim_time: 1.0       # 1.7
  vx_samples: 6       # 3
  vy_samples: 1       # diff drive robot, there is only one sample
  vtheta_samples: 20  # 20

# Trajectory Scoring Parameters
  path_distance_bias: 64.0      # 32.0   - weighting for how much it should stick to the global path plan
  goal_distance_bias: 24.0      # 24.0   - wighting for how much it should attempt to reach its goal
  occdist_scale: 0.5            # 0.01   - weighting for how much the controller should avoid obstacles
  forward_point_distance: 0.325 # 0.325  - how far along to place an additional scoring point
    stop_time_buffer: 0.2         # 0.2    - amount of time a robot must stop in before colliding for a valid traj.
  scaling_speed: 0.25           # 0.25   - absolute velocity at which to start scaling the robot's footprint
  max_scaling_factor: 0.2       # 0.2    - how much to scale the robot's footprint when at speed.

# Oscillation Prevention Parameters
  oscillation_reset_dist: 0.05  # 0.05   - how far to travel before resetting oscillation flags

# Debugging
  publish_traj_pc : true
  publish_cost_grid_pc: true
  global_frame_id: odom


# Differential-drive robot configuration - necessary?
#  holonomic_robot: false

<p style="background:#EE9023;color:white;">Exercise 2.6</p>
<br>
Change the <b>dwa</b> parameter in the local planner parameters file to false in order to use the Trajectory Rollout Algorithm.

<br>
Check if you notice some differences in the performance.
<br>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 2.6</p>

<p style="background:#EE9023;color:white;">End of Exercise 2.6</p>

<p style="background:#EE9023;color:white;">Exercise 2.7</p>
<br>
Execute the command used in the Exercise 1.1, but modify some of this parameters in order to see how they affect to the Local Path Planning.
<br>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 2.7</p>

<p style="background:#EE9023;color:white;">End of Exercise 2.7</p>

In the global planner section we already introduced you to costmaps, focusing on the global costmap. So now it's time to talk about the local costmap. 

### Local Costmap

The first thing you need to know is that the <b>local planner uses the local costmap in order to calculate local plans</b>.
<br><br>
Unlike the global costmap, the local costmap is created directly from the robot's sensors readings. Given a width and a height for the costmap (which are defined by the user), it keeps the robot in the center of the costmap as it moves throughout the environment, dropping obstacle information from the map as the robot moves.

Let's do an exercise so you can get a better a idea of how the local costmap looks, and how to differentiate a local costmap from a global costmap.

<p style="background:#EE9023;color:white;">Exercise 1.6</p>
<br>
a) Open Rviz and add the proper displays in order to visualize the Global and the Local Costmaps.
<br><br>
b) Execute the following command in order to spawn an obstacle in the room.<br>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p><br>
rosrun gazebo_ros spawn_model -file /home/user/object.urdf -urdf -x 0 -y 0 -z 1 -model my_object<br>
</th>
</tr>
</table>

c) Launch the keyboard Teleop and move near the spawned object.
<br>
<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #3</p><br>
roslaunch husky_teleop keyboard_teleop.launch<br>
</th>
</tr>
</table>

d) Check the differences between the Global and the Local Costmap

<p style="background:#AE0202;color:white;">Expected Result for Excercise 1.6</p>

Husky facing the spawned obstacle:

<img src="img/facing_obstacle.png" width="600"></img>

Global Costmap (Obstacle doesn't appears):

<img src="img/global_costmap_obstacle.png" width="600"></img>

Local Costmap (Obstacle does appear):

<img src="img/local_costmap_obstacle.png" width="600"></img>

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 1.6</p>
<br>
Check the following Notes in order to complete the Exercise:
<br><br>
<span style="color:orange">Note 1: </span>You can use the command <i>roslaunch husky_rviz_launchers view_planning.launch</i> in order to launch RViz with predefined displays.<br>
</th>
</tr>
</table>

<p style="background:#EE9023;color:white;">End of Exercise 1.6</p>

So, as you've seen in the previous exercise, the <b>local costmap does detect new objects that appear in the simulation, while the global costmap doesn't</b>. 
<br><br>
This happens, as you may have already deducted, because the global costmap is created from an static map file. This means, the costmap won't change even if the environment does. The local costmap, instead, is created from the robot's sensors readings, so it will always keep updating with the new readings from the sensors.
<br><br>
Since the global costmap and the local costmap don't have the same behavior, the parameters file must also be different. Let's have a look at the most important parameters that we need to set for the local costmap.



### Local Costmap Parameters

The parameters you need to know are the following:
<ul>
<li><b>global_frame</b>: The global frame for the costmap to operate in. In the local costmap, this parameter has to be set to "/odom".</li>
<li><b>static_map </b>: Whether or not to use an static map to initialize the costmap. In the local costmap, this parameter has to be set to "false".</li>
<li><b>rolling_window</b>: Whether or not to use a rolling window version of the costmap. If the static_map parameter is set to true, this parameter must be set to false. In the local costmap, this parameter has to be set to "true".</li>
<li><b>width</b>: The width of the costmap.</li>
<li><b>heigth</b>: The heigth of the costmap.</li>
<li><b>update_frequency</b>: The frequency in Hz for the map to be updated.</li>
<li><b>plugins</b>: Sequence of plugin specifications, one per layer. Each specification is a dictionary with name and type fields. The name is used to define the parameter namespace for the plugin.</li>
</ul>

So, by setting the static_map paramter to false, and the rolling_window parameter to true, we are indicating that we don't want the costmap to be initialized from an static map, but to be built from the robot's sensors readings. Since we won't have a map, the global_frame parameter needs to be changed aswell. Finally, we also need to set a width and a heigth for the costmap, because in this case, it can't get this values from an static map.

<p style="background:#EE9023;color:white;">Exercise 2.8</p>
<br>
Modify width and heigth parameters
<br>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 2.8</p>

<p style="background:#EE9023;color:white;">End of Exercise 2.8</p>

Just as we saw for the global costmap, layers can also be added to the local costmap. In the case of the local costmap, you will usually add these 2 layers:
<ul>
<li><b>costmap_2d::ObstacleLayer</b>: Used for obstacle avoidance.</li>
<li><b>costmap_2d::InflationLayer</b>: Used to inflate obstacles.</li>
</ul>

So you will have something like this:

In [ ]:
plugins:
    - {name: obstacle_layer,      type: "costmap_2d::ObstacleLayer"}
    - {name: inflation_layer,     type: "costmap_2d::InflationLayer"}

As you've already seen through the exercises, the local costmap keeps updating itself . These update cycles are made at a rate specified by the <b>update_frequency</b> parameter. Each cycle works as follows:
<ol>
<li>Sensor data comes in.</li>
<li>Marking and clearing operations are performed.</li>
<li>The appropied cost values are assigned to each cell.</li>
<li>obstacle inflation is performed on each cell with an obstacle. This consists of propagating cost values outwards from each occupied cell out to a specified inflation radius.</li>
</ol>

<p style="background:#EE9023;color:white;">Exercise 1.9</p>
<br>
a) In the local and global costmap parameters files, change the <b>update frequency</b> parameter of the map to be slower.
<br><br>
b) Repeat again the Exercise 1.6, and see what happens now.<br>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 1.9</p>

<p style="background:#EE9023;color:white;">End of Exercise 1.9</p>

Now you may be wondering... what are the marking and clearing operations you mentioned above?
<br><br>
As you already know, the costmap automatically subscribes to the sensor topics and updates itself according to the data it receives from them. Each sensor is used to either mark (insert obstacle information into the costmap), clear (remove obstacle information from the costmap), or both.
<br><br> 
A marking operation is just an index into an array to change the cost of a cell.<br>
A clearing operation, however, cosists of raytracing through a grid from the origin of the sensor outwards for each observation reported.
<br><br>
The marking and clearing operations can be defined in the obstacle layer.

At this point, we can almost say that you already know how to configure both global and local costmaps. But if you remember, there's still a paramters file we haven't talked about. That's the common costmap parameters file. These parameters will affect both the global and the local costmap.

Basically, the parameters you'll have to set in this file are the following:
<ul>
<li><b>footprint</b>: footprint is the contour of the mobile base. In ROS, it is represented by a two dimensional array of the form [x0, y0], [x1, y1], [x2, y2], ...]. This footprint will be used to compute the radius of inscribed circle and circumscribed circle, which are used to inflate obstacles in a way that fits this robot. Usually for safety, we want to have the footprint to be slightly larger than the robot’s real contour.</li>
<br>
<li><b>robot_radius</b>: In the case the robot is circular, we will specify this parameter instead of the footprint.</li>
<br>
<li><b>layers parameters</b>: Here we will define the parameters for each layer.</li>
</ul>

Each layer has its own parameters.

#### Obstacle Layer

The obstacle layer is in charge of the <b>marking and clearing operations</b>.
<br><br>
As you already know, the costmap automatically subscribes to the sensor topics and updates itself according to the data it receives from them. Each sensor is used to either mark (insert obstacle information into the costmap), clear (remove obstacle information from the costmap), or both.
<br><br> 
A marking operation is just an index into an array to change the cost of a cell.<br>
A clearing operation, however, cosists of raytracing through a grid from the origin of the sensor outwards for each observation reported.
<br><br>
The marking and clearing operations can be defined in the obstacle layer.

<ul>
<li><b>max_obstacle_height (default: 2.0)</b>: The maximum height of any obstacle to be inserted into the costmap in meters. This parameter should be set to be slightly higher than the height of your robot.</li>
<li><b>obstacle range (default: 2.5)</b>: The default maximum distance from the robot at which an obstacle will be inserted into the cost map in meters. This can be over-ridden on a per-sensor basis.</li>
<li><b>raytrace_range (default: 3.0)</b>: The default range in meters at which to raytrace out obstacles from the map using sensor data. This can be over-ridden on a per-sensor basis.</li>
<li><b>observation_sources (default: "")</b>: A list of observation source names separated by spaces. This defines each of the <i>source_name</i> namespaces defined below.</li>
</ul>
<br>
Each source_name in observation_sources defines a namespace in which parameters can be set:
<ul>
<li><b>/source_name/topic (default: source_name)</b>: The topic on which sensor data comes in for this source. Defaults to the name of the source.</li>
<li><b>/source_name/data_type (default: "PointCloud")</b>: The data type associated with the topic, right now only "PointCloud", "PointCloud2", and "LaserScan" are supported.</li>
<li><b>/source_name/clearing (default: false)</b>: Whether or not this observation should be used to clear out freespace.</li>
<li><b>/source_name/marking (default: true)</b>: Whether or not this observation should be used to mark obstacles.</li>
<li><b>/source_name/inf_is_valid (default: false)</b>: Allows for Inf values in "LaserScan" observation messages. The Inf values are converted to the laser maximun range.</li>
</ul>

<p style="background:#EE9023;color:white;">Exercise 1.9</p>
<br>
a) In the local and global costmap parameters files, change the <b>update frequency</b> parameter of the map to be slower.
<br><br>
b) Repeat again the Exercise 1.6, and see what happens now.<br>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 1.9</p>

<p style="background:#EE9023;color:white;">End of Exercise 1.9</p>

#### Inflation Layer

The inflation layer is in charge of performing inflation in each cell with an obstacle.

<ul>
<li><b>inflation_radius (default: 0.55)</b>: The radius in meters to which the map inflates obstacle cost values.</li>
<li><b>cost_scaling_factor (default: 10.0)</b>: A scaling factor to apply to cost values during inflation.</li>
</ul>

<p style="background:#EE9023;color:white;">Exercise 1.9</p>
<br>
a) In the local and global costmap parameters files, change the <b>update frequency</b> parameter of the map to be slower.
<br><br>
b) Repeat again the Exercise 1.6, and see what happens now.<br>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 1.9</p>

<p style="background:#EE9023;color:white;">End of Exercise 1.9</p>

#### Static Layer

The static layer is in charge of providing the static map to the costmaps that require it (global costmap).

<ul>
<li><b>map_topic (string, default: "map")</b>: The topic that the costmap subscribes to for the static map.</li>
</ul>

## Recovery Behaviors

It could happen that, while trying to perform a trajectory, the robot gets stuck for some reason. Fortunately, if this happens, the ROS Navigation Stack provides methods that can help your robot to get unstucked and continue navigating. These are the <b>recovery behaviors</b>.
<br><br>
The ROS Navigation Stack provides 2 recovery behaviors: <b>clear costmap</b> and <b>rotate recovery</b>.

In order to enable the recovery behaviors, we need to set the following parameter in the move_base parameters file:
<ul>
<li><b>recovery_behavior_enabled (default: true)</b>: Enables or disables the recovery behaviors.</li>
</ul>

### Rotate Recovery

Bascially, the rotate recovery behavior is a simple recovery behavior that attempts to clear out space by rotating the robot 360 degrees. This way, the robot may be able to find an obstacle-free path to continue navigating.

It has some parameters you can customize in order to change or improve its behavior:

### Rotate Recovery Parameters

<ul>
<li><b>/sim_granularity (double, default: 0.017)</b>: The distance in radians between checks for obstacles when checking if an in-place rotation is safe. Defaults to 1 degree.</li>
<li><b>/frequency (double, default: 20.0)</b>: The frequency in HZ at which to send velocity commands to the mobile base.</li>
</ul>

### Other Parameters

<p style="color:red;"> IMPORTANT: These parameters are already set when using the base_local_planner local planner, they only need to be set explicitly for the recovery behavior if a different local planner is used.</p>

<ul>
<li><b>/yaw_goal_tolerance (double, default: 0.05)</b>: The tolerance in radians for the controller in yaw/rotation when achieving its goal</li>
<li><b>/acc_lim_th (double, default: 3.2)</b>: The rotational acceleration limit of the robot in radians/sec^2</li>
<li><b>/max_rotational_vel (double, default: 1.0)</b>: The maximum rotational velocity allowed for the base in radians/sec</li>
<li><b>/min_in_place_rotational_vel (double, default: 0.4)</b>: The minimum rotational velocity allowed for the base while performing in-place rotations in radians/sec</li>
</ul>

These parameters are set in the move_base parameters file.

<p style="background:#EE9023;color:white;">Exercise 2.9</p>
<br>
Force the robot to move to an obstacle and check if the Rotate Recovery behavior launches.
<br>

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 2.9</p>
<br>
Check the following Notes in order to complete the Exercise:
<br><br>
<span style="color:orange">Note 1:</span>: <br>
<span style="color:orange">Note 2:</span>: <br>
<span style="color:orange">Note 2:</span>: <br>
</th>
</tr>
</table>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 2.9</p>

<p style="background:#EE9023;color:white;">End of Exercise 2.9</p>

### Clear Costmap

The clear costmap recovery is a simple recovery behavior that clears out space by clearing obstacles outside of a specified region from the robot's map. Basically, it reverts the local costmap to have the same state as the global costmap.

The radius away from the robot (in meters) which obstacles will be removed from the costmap when they are reverted to the static map, can be setted by modifying the next parameter:
<ul>
<li><b>/reset_distance</b></li>
</ul>

This parameter is set in the move_base parameters file. Go back to the move_base section in order to refresh it.

<p style="background:#EE9023;color:white;">Exercise 3.0</p>
<br>
Modify reset_distance parameter
<br>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 3.0</p>

<p style="background:#EE9023;color:white;">End of Exercise 3.0</p>

## Oscillation Supression

Oscillation occur when in either of the x, y, or theta dimensions, positive and negative values are chosen consecutively.
<br><br>
To prevent oscillations, when the robot moves in any direction, for the next cycles the opposite direction is marked invalid, until the robot has moved beyond a certain distance from the position where the flag was set.

In order to manage this issue, there exist 2 parameter that you can set in the move_base parameters file.

<ul>
<li><b>oscillation_timeout (double, default: 0.0)</b>: How long in seconds to allow for oscillation before executing recovery behaviors. A value of 0.0 corresponds to an infinite timeout.</li>
<li><b>oscillation_distance (double, default: 0.5)</b>: How far in meters the robot must move to be considered not to be oscillating. Moving this far resets the timer counting up to the ~oscillation_timeout</li>
</ul>

<p style="background:#EE9023;color:white;">Exercise 3.0</p>
<br>
Modify oscillation_distance parameter
<br>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 3.0</p>

<p style="background:#EE9023;color:white;">End of Exercise 3.0</p>

## Recap

Congratulations! At this point you've already seen almost all the important parts that this Chapter covers. And since this is the last chapter of the course, this means that you are very close to completely know how to deal with ROS Navigation!
<br><br>
Anyways, you may be overwhelmed with all the information you've got about Path Planning. That's why I think this is a good moment to do a summary of all you've seen in this chapter until now. Let's begin!

### The move_base node

The move_base node is, basically, the node that coordinates all the Path Planning System. It takes a goal pose as input, and outputs the necessary velocity commands in order to move the robot from an initial pose to the specified goal pose. In order to achieve this, the move_base node manages a whole internal process where it take place different parts:
<ul>
<li>global planner</li>
<li>local planner</li>
<li>costmaps</li>
<li>recovery behaviors</li>
</ul>

### The global planner

When a new goal is received by the move_base node, it is immediately sent to the global planner. The global planner, then, will calculate a safe path for the robot to arrive to the specified goal. The global planner uses the global costmap data in order to calculate this path.
<br><br>
There exist different types of global planners. Depending on your setup, you will use one or another.

### The local planner

Once the global planner has calculated a path for the robot, this is sent to the local planner. The local planner, then, will execute this path breaking it into smaller (local) parts. So, given a plan to follow and a map, the local planner will provide velocity commands in order to move the robot. The local planner operates over a local costmap.
<br><br>
There exist different types of local planners. Dependng on the kind of performance you require, you will use one or another.

### Costmaps

Costmaps are, basically, maps that represent which points of the map are safe for the robot to be in, and which ones are not. There exist 2 types of costmaps:
<ul>
<li>global costmap</li>
<li>local costmap</li>
</ul>
<br>
Basically, the difference between them is that the global costmap is built using the data from a previously built static map while the local costmap is built from the robot's sensors readings.

### Recovery Behaviors

The recovery behaviors provide methods to the robot in case it get's stucked. The Navigation Stack provides 2 different recovery behaviors:
<ul>
<li>rotate recovery</li>
<li>clear costmap</li>
</ul>

### Configuration

Since there are lots of different nodes working together, the number of parameters available in order to configure the different nodes is also very high. I think it would be a great idea if we also summarize the different parameter files that we will need to set for Path Planning. The parameter files you'll need are the following:
<ul>
<li><b>move_base_params.yaml</b></li>
<li><b>navfn_global_planner_params.yaml</b></li>
<li><b>dwa_local_planner_params.yamls</b></li>
<li><b>common_costmap_params.yaml</b></li>
<li><b>global_costmap_params.yaml</b></li>
<li><b>local_costmap_params.yaml</b></li>
</ul>


Besides the parameter files shown above, we will also need to have a lanch file in order to launch the whole system and load the different parameters.

### Overall

After getting the current position of the robot, we can send a goal position to the <b>move_base</b> node. This node will then send this goal position to a <b>global planner</b> which will plan a path from the current robot position to the goal position. This plan is with respect to the <b>global costmap</b>, which is feeding from the <b>map server</b>.
<br><br>
The <b>global planner</b> will then send this path to the <b>local planner</b>, which executes each segment of the global plan. The <b>local planner</b> gets the odometry and the laser data values and finds a collision free local plan for the robot. The <b>local planner</b> is associated with the <b>local costmap</b>, which can monitor the obstacle(s) around the robot. The <b>local planner</b> generates the velocity commands and sends them to the base controller. The robot base controller will then convert this commands into real robot movement.
<br><br>
If the robot is stuck somewhere, the recovery behavior nodes, such as the <b>clear costmap recovery</b> or <b>rotate recovery</b>, will be called.

Wow!! That sounds amazing!! Ok, so now that we have a better picture of the whole thing, let's see what really happened in the previous exercise.
<br>
Bascially, you sent a goal pose to the <i>move_base_node/goal</i> topic using the 2D Nav Goal tool of Rviz. Then, the move_base node received this goal pose, and and started the whole process described just above. Now everything makes more sense, right?

## Dynamic Reconfigure

Until now we've seen how to change parameters by modifying them in the parameters files. But, guess what... this is not the only way you can change parameters! You can also change dynamic parameters by using the rqt_reconfigure tool. Follow the next steps:

<p style="background:#EE9023;color:white;">Exercise 2.7</p>
<br>
a) Run the next command in order to open the rqt_reconfigure tool.
<br>
<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #2</p><br>
rosrun rqt_reconfigure rqt_reconfigure<br>
</th>
</tr>
</table>

b) Open the move_base group.
<br><br>
c)Select the DWAPlannerROS node.
<br><br>
d) Set the <b>pdist_scale</b> parameter to something high like 2.5.
<br><br>
e) Open Rviz and visualize the Global and Local Plans.

<table style="float:left;">
<tr>
<th>
<p style="background:#3B8F10;color:white;">Data for Exercise 2.7</p>
<br>
Check the following Notes in order to complete the Exercise:
<br><br>
<span style="color:orange">Note 1:</span>: <br>
<span style="color:orange">Note 2:</span>: <br>
<span style="color:orange">Note 2:</span>: <br>
</th>
</tr>
</table>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 2.7</p>

<img src="img/rqt_reconfigure.png" width="600"></img>

<p style="background:#EE9023;color:white;">End of Exercise 2.7</p>

## Other Useful Visualizations in Rviz

Until now we've already seen some ways of visualizing different parts of the move_base node process throgu Rviz. But there are a couple more that may be interesting to know:

### Robot Footprint

It shows the footprint of the robot.

<img src="img/footprint.png" width="700"></img>

### Current Goal

To show the goal pose that the navigation stack is attempting to achieve add a Pose Display and set its topic to /move_base_simple/goal. You will now be able to see the goal pose as a red arrow. It can be used to know the final position of the robot.

<img src="img/current_goal.png" width="700"></img>

<p style="background:#EE9023;color:white;">Exercise 3.1</p>
<br>
Open Rviz and try to visualize the elements described above.
<br>

<p style="background:#AE0202;color:white;">Expected Result for Excercise 3.1</p>

<p style="background:#EE9023;color:white;">End of Exercise 3.1</p>

## CONCLUSIONS

<img src="img/overview_tf_small.png" width="800"></img>